### 의존성 설치

In [ ]:
%pip install -r requirements.txt

### MongoDB 연결

In [ ]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from dotenv import load_dotenv
import os

# 환경 변수 로드
load_dotenv()

USERNAME = os.getenv("MONGO_USERNAME")
PASSWORD = os.getenv("MONGO_PASSWORD")
HOST = os.getenv("MONGO_HOST")
PORT = int(os.getenv("MONGO_PORT"))

url = f"mongodb://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/"

# MongoDB 연결
try:
    client = MongoClient(url)
    client.admin.command('ping')
    print("Successfully connected to MongoDB!")
except ConnectionFailure as e:
    print(f"MongoDB connection failed: {e}")

db = client['s307_db']
collection = db['s307_collection']

### PDF 재생성 설정

In [ ]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import os
import platform

# MongoDB의 root 객체에서 페이지 크기 정보 가져오기
root_data = collection.find_one({"page_num": 0})

# parsing.py에서 저장한 width/height 사용
if 'width' in root_data and 'height' in root_data:
    page_width = root_data['width']
    page_height = root_data['height']
else:
    # Fallback to A4 if dimensions not found
    page_width, page_height = A4
    print("Warning: Page dimensions not found in root document, using A4")

page_count = root_data['page_count']

# 디버깅
print(f"PDF Info: {page_width} x {page_height}, Total Pages: {page_count}")

# OS별 폰트 매핑 설정
system = platform.system()

# 기본 폰트 정의 (Fallback용 - Helvetica는 한글 미지원하므로 한글 폰트 사용)
DEFAULT_FONT = 'MalgunGothic-Fallback'

if system == 'Windows':
    font_mapping = {
        'DotumChe': {
            'normal': 'C:/Windows/Fonts/gulim.ttc',
            'bold': 'C:/Windows/Fonts/malgunbd.ttf',
        },
        'Dotum': {
            'normal': 'C:/Windows/Fonts/gulim.ttc',
            'bold': 'C:/Windows/Fonts/malgunbd.ttf',
        },
        'Malgun': {
            'normal': 'C:/Windows/Fonts/malgun.ttf',
            'bold': 'C:/Windows/Fonts/malgunbd.ttf',
        },
    }
    default_font_path = 'C:/Windows/Fonts/malgun.ttf'

elif system == 'Darwin':  # macOS
    font_mapping = {
        'DotumChe': {
            'normal': '/System/Library/Fonts/Supplemental/AppleGothic.ttf',
            'bold': '/System/Library/Fonts/Supplemental/AppleGothic.ttf',
        },
        'Dotum': {
            'normal': '/System/Library/Fonts/Supplemental/AppleGothic.ttf',
            'bold': '/System/Library/Fonts/Supplemental/AppleGothic.ttf',
        },
        'Malgun': {
            'normal': '/System/Library/Fonts/Supplemental/AppleGothic.ttf',
            'bold': '/System/Library/Fonts/Supplemental/AppleGothic.ttf',
        },
    }
    default_font_path = '/System/Library/Fonts/Supplemental/AppleGothic.ttf'

else:  # Linux
    font_mapping = {
        'DotumChe': {
            'normal': '/usr/share/fonts/truetype/nanum/NanumGothic.ttf',
            'bold': '/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf',
        },
        'Dotum': {
            'normal': '/usr/share/fonts/truetype/nanum/NanumGothic.ttf',
            'bold': '/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf',
        },
        'Malgun': {
            'normal': '/usr/share/fonts/truetype/nanum/NanumGothic.ttf',
            'bold': '/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf',
        },
    }
    default_font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'

# 기본 폰트 등록
try:
    pdfmetrics.registerFont(TTFont(DEFAULT_FONT, default_font_path))
    default_font = DEFAULT_FONT
    print(f"Default font registered: {DEFAULT_FONT} from {default_font_path} (OS: {system})")
except Exception as e:
    # 폰트 등록 실패 시에도 DEFAULT_FONT 사용 (나중에 등록 시도)
    default_font = DEFAULT_FONT
    print(f"Warning: Failed to register {DEFAULT_FONT}, will retry on first use (OS: {system})")

# 이미 등록된 폰트 추적
registered_fonts = {}

### 폰트 등록 함수

**DB 폰트명을 동적으로 분석하여 적절한 시스템 폰트 등록**

In [ ]:
def register_font_from_db(fontname):
    """
    DB 폰트명을 분석하여 적절한 폰트 파일로 등록
    
    Args:
        fontname: PDF의 폰트명 (예: "FAAAAH+DotumChe,Bold")
    
    Returns:
        등록된 폰트 이름
    """
    # 이미 등록된 폰트이거나 폰트명이 없으면
    if not fontname or fontname in registered_fonts:
        return registered_fonts.get(fontname, default_font)
    
    # DB 폰트명 분석
    # 예: "FAAAAH+DotumChe,Bold" -> base="DotumChe", is_bold=True
    parts = fontname.split('+')[-1]  # Subset 접두어 제거 (FAAAAH+)
    is_bold = 'Bold' in parts or 'bold' in parts
    base_font = parts.split(',')[0]  # 베이스 폰트명 추출
    
    # 매핑된 폰트 파일 찾기
    font_file = None
    for key in font_mapping:
        if key in base_font:
            if is_bold and 'bold' in font_mapping[key]:
                font_file = font_mapping[key]['bold']
            else:
                font_file = font_mapping[key]['normal']
            break
    
    # 폰트 파일 등록
    if font_file and os.path.exists(font_file):
        try:
            pdfmetrics.registerFont(TTFont(fontname, font_file))
            registered_fonts[fontname] = fontname
            return fontname
        except:
            # 등록 실패 시 기본 폰트 사용
            registered_fonts[fontname] = default_font
            return default_font
    else:
        # 폰트 파일을 찾지 못한 경우 기본 폰트 사용
        registered_fonts[fontname] = default_font
        return default_font

### MongoDB 데이터로 PDF 재생성

**중요**: pdfplumber와 reportlab 모두 **PDF 좌표계(bottom-up)**를 사용하므로, y0 좌표를 변환 없이 직접 사용합니다.

- **chars**: character 단위 텍스트 (y0 직접 사용, DB 폰트명 기반 매칭)
- **lines**: 선 객체 (y0/y1 직접 사용)
- **rects**: 사각형 객체 (y0 직접 사용)
- **images**: 이미지 객체 (y0 직접 사용, URL 또는 base64 지원)

In [ ]:
def reconstruct_pdf_from_mongodb(output_filename="reconstructed.pdf"):
    """MongoDB에 저장된 문자, 선, 사각형, 이미지 정보로 PDF 재생성"""
    
    # PDF Canvas 생성
    c = canvas.Canvas(output_filename, pagesize=(page_width, page_height))
    
    # DB에서 페이지별로 데이터 가져오기 (page_num > 0인 문서만, 정렬)
    pages_data = collection.find({"page_num": {"$gt": 0}}).sort("page_num", 1)
    
    for page_data in pages_data:
        page_num = page_data['page_num']
        print(f"Processing page {page_num}...")
        
        # 1. Rects 먼저 그리기 (배경)
        rects = page_data.get('rects', [])
        for rect in rects:
            x0 = rect['x0']
            y0 = rect['y0']  # PDF 좌표 (bottom-up) - 직접 사용!
            width = rect.get('width', 0)
            height = rect.get('height', 0)
            linewidth = rect.get('linewidth', 1)
            stroke = rect.get('stroke', True)
            fill = rect.get('fill', False)
            stroke_color = rect.get('stroking_color')
            fill_color = rect.get('non_stroking_color')
            
            # 선 굵기 설정
            c.setLineWidth(linewidth)
            
            # 선 색상 설정
            if stroke_color is not None:
                if isinstance(stroke_color, (list, tuple)) and len(stroke_color) >= 3:
                    c.setStrokeColorRGB(stroke_color[0], stroke_color[1], stroke_color[2])
                elif isinstance(stroke_color, (int, float)):
                    c.setStrokeGray(stroke_color)
            
            # 채우기 색상 설정
            if fill_color is not None:
                if isinstance(fill_color, (list, tuple)) and len(fill_color) >= 3:
                    c.setFillColorRGB(fill_color[0], fill_color[1], fill_color[2])
                elif isinstance(fill_color, (int, float)):
                    c.setFillGray(fill_color)
            
            # 사각형 그리기
            if fill and stroke:
                c.rect(x0, y0, width, height, stroke=1, fill=1)
            elif fill:
                c.rect(x0, y0, width, height, stroke=0, fill=1)
            elif stroke:
                c.rect(x0, y0, width, height, stroke=1, fill=0)
        
        # 2. Lines 그리기
        lines = page_data.get('lines', [])
        for line in lines:
            x0 = line['x0']
            x1 = line['x1']
            y0_pdf = line['y0']  # PDF 좌표 (bottom-up) - 직접 사용!
            y1_pdf = line['y1']
            linewidth = line.get('linewidth', 1)
            stroke_color = line.get('stroking_color')
            
            c.setLineWidth(linewidth)
            
            if stroke_color is not None:
                if isinstance(stroke_color, (list, tuple)) and len(stroke_color) >= 3:
                    c.setStrokeColorRGB(stroke_color[0], stroke_color[1], stroke_color[2])
                elif isinstance(stroke_color, (int, float)):
                    c.setStrokeGray(stroke_color)
            
            c.line(x0, y0_pdf, x1, y1_pdf)
        
        # 3. Images 그리기 (URL 방식 지원)
        images = page_data.get('images', [])
        if images:
            import base64
            import io
            from PIL import Image
            
            for img_data in images:
                x0 = img_data['x0']
                y0 = img_data['y0']  # PDF 좌표 (bottom-up) - 직접 사용!
                width = img_data['width']
                height = img_data['height']
                
                pil_image = None
                
                # Method 1: URL 방식 (parsing.py)
                if 'url' in img_data and img_data['url']:
                    try:
                        image_path = img_data['url']
                        if os.path.exists(image_path):
                            pil_image = Image.open(image_path)
                        else:
                            print(f"Warning: Image file not found: {image_path}")
                    except Exception as e:
                        print(f"Warning: Failed to load image from {img_data.get('url')}: {e}")
                
                # Method 2: base64 방식 (대안)
                elif 'image_data' in img_data:
                    try:
                        image_bytes = base64.b64decode(img_data['image_data'])
                        pil_image = Image.open(io.BytesIO(image_bytes))
                    except Exception as e:
                        print(f"Warning: Failed to decode base64 image: {e}")
                
                # 이미지 그리기
                if pil_image:
                    try:
                        c.drawInlineImage(pil_image, x0, y0, width, height, preserveAspectRatio=False)
                    except Exception as e:
                        print(f"Warning: Failed to draw image at ({x0}, {y0}): {e}")
        
        # 4. Chars 마지막에 그리기 (전경) - DB 폰트명 기반 매칭 사용
        chars = page_data.get('chars', [])
        for char in chars:
            text = char['text']
            x0 = char['x0']
            y0 = char['y0']  # PDF 좌표 (bottom-up) - 직접 사용!
            
            # 폰트 정보 가져오기
            fontname = char.get('fontname', None)
            font_size = char.get('size', 10)
            
            # 색상 정보 가져오기
            non_stroking_color = char.get('non_stroking_color', None)
            stroking_color = char.get('stroking_color', None)
            
            try:
                # 폰트 설정 - register_font_from_db 사용
                if fontname:
                    font_to_use = register_font_from_db(fontname)
                    c.setFont(font_to_use, font_size)
                else:
                    c.setFont(default_font, font_size)
                
                # 텍스트 색상 설정 (non_stroking_color = 텍스트 내부 색상)
                if non_stroking_color is not None:
                    if isinstance(non_stroking_color, (tuple, list)):
                        if len(non_stroking_color) == 3:
                            c.setFillColorRGB(*non_stroking_color)
                        elif len(non_stroking_color) == 1:
                            c.setFillGray(non_stroking_color[0])
                    elif isinstance(non_stroking_color, (int, float)):
                        c.setFillGray(non_stroking_color)
                else:
                    # 기본값: 검정색
                    c.setFillColorRGB(0, 0, 0)
                
                # 텍스트 외곽선 색상 설정
                if stroking_color is not None:
                    if isinstance(stroking_color, (tuple, list)):
                        if len(stroking_color) == 3:
                            c.setStrokeColorRGB(*stroking_color)
                        elif len(stroking_color) == 1:
                            c.setStrokeGray(stroking_color[0])
                    elif isinstance(stroking_color, (int, float)):
                        c.setStrokeGray(stroking_color)
                
                # 텍스트 그리기 - 특수문자 fallback 포함
                try:
                    c.drawString(x0, y0, text)
                except Exception as inner_e:
                    # 렌더링 실패 시 특수문자 대체 시도
                    # ∙ (U+2219) → · (U+00B7) 또는 • (U+2022)
                    fallback_text = text.replace('\u2219', '\u00B7').replace('\u2219', '\u2022')
                    try:
                        c.drawString(x0, y0, fallback_text)
                    except:
                        # 최종 실패 시 건너뛰기
                        continue
                        
            except:
                continue  # 렌더링 실패 시 건너뛰기
        
        # 다음 페이지로
        c.showPage()
    
    # PDF 저장
    c.save()
    print(f"PDF saved to: {output_filename}")

# PDF 재생성 실행
reconstruct_pdf_from_mongodb("reconstructed.pdf")

### 연결 종료

In [ ]:
client.close()
print("MongoDB connection closed successfully!")